<a href="https://colab.research.google.com/github/ChristineLong/document/blob/main/scanned_doc_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [ ]:
import tensorflow as tf
from keras import backend as k
import shutil
import os
import tarfile
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing
import codecs# this is used for file operations 
import random as r
import time
import math
import cv2
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import log_loss,confusion_matrix
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,MaxPooling2D,Conv2D,Dropout,BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing import image
from sklearn.datasets import load_files
import cv2
import pickle
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,precision_score,recall_score
from statistics import mode
from sklearn.utils.multiclass import unique_labels
# Don't pre-allocate memory; allocate as-needed
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

from keras.callbacks import ReduceLROnPlateau
from keras import regularizers

# Import data

In [ ]:
# Code to unzip the raw data
file_path="rvl-cdip.tar.gz"


import tarfile
tar = tarfile.open(file_path)
tar.extractall()
tar.close()

# Preprocess

In [ ]:
classes = {'0': 'letter',
 '1': 'form',
 '10': 'budget',
 '11': 'invoice',
 '12': 'presentation',
 '13': 'questionnaire',
 '14': 'resume',
 '15': 'memo',
 '2': 'email',
 '3': 'handwritten',
 '4': 'advertisement',
 '5': 'scientific report',
 '6': 'scientific publication',
 '7': 'specification',
 '8': 'file folder',
 '9': 'news article'}

label_path='labels/'

# This function will create an index file with image paths and labels in csv format
# obtained csv will have two columns path(path to acess the image),label(class of the image)
def create_csv(file_path):
    images=[]
    labels=[]
    label_file=pd.DataFrame(columns=['path','label'],index=None)
    file=open(file_path)
    for ele in file:
        images.append(ele.split(' ')[0])
        labels.append(ele.split(' ')[1].rstrip())
    label_file.path=images
    label_file.label=labels
    return label_file

train_label_path=label_path+'train.txt' 
test_label_path=label_path+'test.txt' 
val_label_path=label_path+'val.txt'
# creating index file for train,test and val dataset
train_label=create_csv(train_label_path)
test_label=create_csv(test_label_path)
val_label=create_csv(val_label_path)
    
# Creating train,validation and Test directories
os.mkdir('train')
os.mkdir('test')
os.mkdir('val')
# run this cell to enable intra domain transfer learning
regions = ['Whole','header', 'footer', 'left_body', 'right_body']
for region in regions:
    os.mkdir('train/'+region)
    os.mkdir('val/'+region)
classes_name=[v for (k,v) in classes.items()]
for region in regions:
    for label in classes_name:
        os.mkdir('train/'+region+'/'+label)
        os.mkdir('val/'+region+'/'+label)
# Moving files for test directory from common directory
folders=[classes[str(i)] for i in range(16)]
for folder in folders:
    if(os.path.exists('test/'+folder)== False):
        os.mkdir('test/'+folder)
for i in tqdm(range(len(test_label))):
    if(os.path.exists('images/'+test_label.iloc[i].path)):
        source='images/'+test_label.iloc[i].path
        dest='test/'+classes[str(test_label.iloc[i].label)]
        if(os.path.exists(dest+'/'+source.split('/')[-1])==False):
            shutil.move(source,dest)
 
# # Moving files for train directory from common directory
for i in tqdm(range(len(train_label))):
    if(os.path.exists('images/'+train_label.iloc[i].path)):
        source='images/'+train_label.iloc[i].path
        dest='train/Whole/'+classes[str(train_label.iloc[i].label)]
        if(os.path.exists(dest+'/'+source.split('/')[-1])==False):
            shutil.move(source,dest)

#  Moving files for validation directory from common directory
for i in tqdm(range(len(val_label))):
    if(os.path.exists('images/'+val_label.iloc[i].path)):
        source='images/'+val_label.iloc[i].path
        dest='val/Whole/'+classes[str(val_label.iloc[i].label)]
        if(os.path.exists(dest+'/'+source.split('/')[-1])==False):
            shutil.move(source,dest)

val_dir='val/Whole'
count,limit=0,2000
for folder in tqdm(os.listdir(val_dir)):
  #os.shuffle(val_dir+'/'+folder)
  for image in os.listdir(val_dir+'/'+folder):
        img = cv2.imread(val_dir+'/'+folder+'/'+image,0)
        height = img.shape[0]
        width = img.shape[1]
        header = img[0:(int(height*0.33)), 0:width]
        footer = img[int(height*0.67):height, 0:width]
        left_body = img[int(height*0.33):int(height*0.67), 0:int(width*0.5)]
        right_body = img[int(height*0.33):int(height*0.67), int(width*0.5):width]
        header_path='val/header/'+folder+'/'+image.split('/')[-1]
        footer_path='val/footer/'+folder+'/'+image.split('/')[-1]
        left_path='val/left_body/'+folder+'/'+image.split('/')[-1]
        right_path='val/right_body/'+folder+'/'+image.split('/')[-1]
        if(os.path.exists(header_path)==False):
            cv2.imwrite(header_path, header)
        if(os.path.exists(footer_path)==False):
            cv2.imwrite(footer_path, footer)
        if(os.path.exists(left_path)==False):
            cv2.imwrite(left_path, left_body)
        if(os.path.exists(right_path)==False):
            cv2.imwrite(right_path, right_body)
        count+=1
        if(count==limit):
            break

train_dir='train/Whole'
count,limit=0,2000
for folder in tqdm(os.listdir(train_dir)):
  #os.shuffle(train_dir+'/'+folder)
  count=0
    for image in os.listdir(train_dir+'/'+folder):
        img = cv2.imread(train_dir+'/'+folder+'/'+image,0)
        height = img.shape[0]
        width = img.shape[1]
        header = img[0:(int(height*0.33)), 0:width]
        footer = img[int(height*0.67):height, 0:width]
        left_body = img[int(height*0.33):int(height*0.67), 0:int(width*0.5)]
        right_body = img[int(height*0.33):int(height*0.67), int(width*0.5):width]
        header_path='train/header/'+folder+'/'+image.split('/')[-1]
        footer_path='train/footer/'+folder+'/'+image.split('/')[-1]
        left_path='train/left_body/'+folder+'/'+image.split('/')[-1]
        right_path='train/right_body/'+folder+'/'+image.split('/')[-1]
        if(os.path.exists(header_path)==False):
            cv2.imwrite(header_path, header)
        if(os.path.exists(footer_path)==False):
            cv2.imwrite(footer_path, footer)
        if(os.path.exists(left_path)==False):
            cv2.imwrite(left_path, left_body)
        if(os.path.exists(right_path)==False):
            cv2.imwrite(right_path, right_body)
        count+=1
        if(count==limit):
            break

#Train

In [ ]:
# Training configuaration for CNN- to be trained on whole images
img_width, img_height = 224,224
train_dir = "train/Whole"
val_dir = "val/Whole"
test_dir="test"
nb_train_samples = 320000
nb_validation_samples = 40000 
batch_size = 50
epochs = 20
num_classes=16
log=[]

In [ ]:
# preparing Training and Validation data using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, featurewise_std_normalization=True)
val_datagen = ImageDataGenerator(rescale = 1./255, featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size, 
                                                    class_mode = "categorical")

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size = (img_height, img_width),
                                                batch_size = batch_size, 
                                                class_mode = "categorical")

# Importing keras InceptionRsnetv2 pretrained model (on ImageNet)
# We will use the ImageNet weight to initialize the model and fine tune using backpropagation(Transfer Learning) 

model=applications.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#for layer in model.layers[:15]:
#    layer.trainable = False
#Adding custom Layers 
x = model.output
x = Dropout(0.5)(x)
x = Flatten()(x)
output = Dense(num_classes, activation="softmax")(x)
model = Model(input = model.input, output = output)

#model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
checkpoint = ModelCheckpoint("inceptionresnet.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=3, min_lr=0.0001,mode='auto')
callbacks=[checkpoint,reduce_lr]
history=model.fit_generator(train_generator,
                            samples_per_epoch =nb_train_samples//batch_size,
                            epochs = epochs,
                            validation_data = val_generator,
                            validation_steps =math.ceil(nb_validation_samples//(batch_size)),
                            callbacks = callbacks,verbose=1)
log.append(history)
model.save('Inceptionresnet_final.h5')

model=applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#for layer in model.layers[:15]:
#    layer.trainable = False
#Adding custom Layers 
x = model.output
x = Flatten()(x)
output = Dense(num_classes, activation="softmax")(x)
model = Model(input = model.input, output = output)
checkpoint = ModelCheckpoint("vgg16.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=3, min_lr=0.0001,mode='auto')
callbacks=[checkpoint,reduce_lr]
history=model.fit_generator(train_generator,
                            samples_per_epoch =nb_train_samples//batch_size,
                            epochs = epochs,
                            validation_data = val_generator,
                            validation_steps = math.ceil(nb_validation_samples//(batch_size)),
                            callbacks = callbacks,verbose=1)
model.save('vgg16_final.h5')

# Evaluate

In [ ]:
model1=load_model('vgg16.h5')
model2=load_model('resinception.h5')
img_width, img_height = 224,224
train_dir = "train"
val_dir = "val"
test_dir="test"
nb_train_samples = 300000
nb_validation_samples = 32000
batch_size = 1280
epochs = 20
num_classes=16
log=[]
# Test Data generator to generate test data in same format of training data
test_datagen = ImageDataGenerator(
rescale = 1./255,
featurewise_std_normalization=True)
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size = (img_height, img_width),
batch_size = batch_size,shuffle='True',
class_mode = "categorical")
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,precision_score,recall_score
from statistics import mode
from sklearn.utils.multiclass import unique_labels
def eval_model(model1,model2,test_generator,limit):
    """
    This function creates an ensamble of candidate CNN models and predict the result
    model1,model2,model3 -- candidate models
    test_generator -- Instance of ItemDataGenerator to randomly generate test data
    limit -- limits the number of test samples
    
    """
    try:
         x,y=test_generator.next()
    except:
        print('')
    y_pred=[]
    y_true=[]
    count=0
    #print(len(x))
    for i in range(len(x)):
        #pred1=np.argmax(model1.predict(np.expand_dims(x[i], axis=0)))
        #pred2=np.argmax(model2.predict(np.expand_dims(x[i], axis=0)))
        #pred3=np.argmax(model3.predict(np.expand_dims(x[i], axis=0)))
        try:
            # If there are clear majority in prediction add the majority voted class
            y_pred.append(np.argmax(model1.predict(np.expand_dims(x[i], axis=0))+model2.predict(np.expand_dims(x[i], axis=0))))
        except:
            # else select prediction of second model 
            y_pred.append(pred2)
        y_true.append(np.argmax(y[i]))
        count+=1
        # limit the number of samples to be considered
        if(count==limit):
            break
    return y_pred,y_true

def get_result(y_pred,y_true):
    """
    This function evaluate the performance(accuracy,f1-score,precision,recall,confusion matrix of the network).
    """
    print('-----Model Evaluation------')
    accuracy=np.round(accuracy_score(y_pred,y_true),3)
    f1=np.round(f1_score(y_pred,y_true,average='macro'),3)
    precision=np.round(precision_score(y_pred,y_true,average='macro'),3)
    recall=np.round(recall_score(y_pred,y_true,average='macro'),3)
    print('Accuracy:',accuracy*100,'%')
    print('Macro F1 Score:',f1)
    print('Precision Score:',precision)
    print('Recall Score:',recall)
    #plot_confusion_matrix(y_true,y_pred)
    return accuracy,f1,precision,recall
    
y_pred,y_true=eval_model(model1,model1,test_generator,30000)
accuracy,f1,precision,recall=get_result(y_pred,y_true)	

Full repo: https://github.com/arpan65/Scanned-document-classification-using-deep-learning/tree/master/scripts